In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
from tqdm import tqdm
import os

In [2]:
url_tzkt = "https://api.tzkt.io/v1/"

tz_start_date = "2018-07-03"
tz_end_date = "2023-06-27"

tz_start_timestamp = "2018-07-03T00:00:01Z"
tz_end_timestamp = "2023-06-27T23:59:59Z"

dest_dir = "/mnt/Ivana/Data/Tezos/Raw_data/"


In [4]:
# Flatten the nested json elements

def normalize_json(data: dict) -> dict:
  
    new_data = dict()
    for key, value in data.items():
        if not isinstance(value, dict):
            new_data[key] = value
        else:
            for k, v in value.items():
                new_data[key + "_" + k] = v
  
    return new_data

def normalize_json_list(json_list: list) -> list:
    result = []
    for j in json_list:
        result.append(normalize_json(j))

    return result



## Get Daily Statistics

In [50]:
params = {
    "date.gt": tz_start_date, 
    "limit" : 10000
}
stats_url = url_tzkt + "statistics/daily"

response = json.loads(requests.get(url = stats_url, params = params).text)
df = pd.DataFrame.from_dict(response)

date_format = '%Y-%m-%dT%H:%M:%SZ'
df["timestamp"] = df.timestamp.apply(lambda x: datetime.strptime(x, date_format))
df["date"] = df.timestamp.apply(lambda x: x.date())

df.drop(["cycle", "totalVested"], axis = 1, inplace=True)
df.rename({"level":"blockHeight"}, axis=1, inplace=True)

In [52]:
df.to_csv("Tezos/TZ_daily_statistics.csv", index=False)

In [48]:
df.head()

,date,level,timestamp,totalSupply,circulatingSupply,totalBootstrapped,totalCommitments,totalActivated,totalCreated,totalBurned,totalBanished,totalFrozen,totalRollupBonds,totalSmartRollupBonds,totalVested,cycle
0,2018-07-03,4679,2018-07-03 23:59:12,764318316621700,270417439431697,156020221896628,608298376116072,114420701250452,16000000,297391000,0,23202324383,0,0,0,NaN
1,2018-07-04,6118,2018-07-04 23:59:27,764318234135700,278140375571381,156020221896628,608298376116072,122200972177404,16000000,379877000,0,80454625651,0,0,0,NaN
2,2018-07-05,7549,2018-07-05 23:59:12,764318169371700,292856901571425,156020221896628,608298376116072,136974595855972,16000000,444641000,0,137487540175,0,0,0,NaN
3,2018-07-06,8977,2018-07-06 23:59:42,764318141424700,301843209674206,156020221896628,608298376116072,146048927625043,32000000,488588000,0,225483259465,0,0,0,NaN
4,2018-07-07,10408,2018-07-07 23:59:27,764318097220700,306897528317219,156020221896628,608298376116072,151216986592732,32000000,532792000,0,339179380141,0,0,0,NaN


## Get Blocks

In [ ]:
params = {
    "timestamp.gt": tz_start_timestamp,
    "timestamp.lt": tz_end_timestamp,
    "limit" : 10000
}

blocks_url = url_tzkt + "blocks"
i = 0

final_result = []
while(True):
    params["offset"] = 10000 * i
    response = json.loads(requests.get(url = blocks_url, params = params).text)
    
    i += 1
    if len(response) == 0:
        break

    print(len(final_result))
    final_result.extend(response)

In [20]:
# Check whether all entries have the same keys
all_keys = set()

for i in final_result:
    all_keys.add(tuple(i.keys()))

for a in all_keys:
    print(a)

('cycle', 'level', 'hash', 'timestamp', 'proto', 'payloadRound', 'blockRound', 'validations', 'deposit', 'reward', 'bonus', 'fees', 'nonceRevealed', 'proposer', 'producer', 'software', 'lbToggle', 'lbToggleEma', 'priority', 'baker', 'lbEscapeVote', 'lbEscapeEma')
('cycle', 'level', 'hash', 'timestamp', 'proto', 'payloadRound', 'blockRound', 'validations', 'deposit', 'reward', 'bonus', 'fees', 'nonceRevealed', 'proposer', 'producer', 'software', 'lbToggleEma', 'priority', 'baker', 'lbEscapeVote', 'lbEscapeEma')
('cycle', 'level', 'hash', 'timestamp', 'proto', 'payloadRound', 'blockRound', 'validations', 'deposit', 'reward', 'bonus', 'fees', 'nonceRevealed', 'proposer', 'producer', 'lbToggleEma', 'priority', 'baker', 'lbEscapeVote', 'lbEscapeEma', 'software')


In [19]:
# Add missing keys to certains jsons
for i in final_result:
    if "software" not in i.keys():
        i["software"] = None
    if "software" not in i.keys():
        i["lbToggle"] = None

In [24]:
normalized = normalize_json_list(final_result)
df_blocks = pd.DataFrame.from_dict(normalized)
df_blocks

,cycle,level,hash,timestamp,proto,payloadRound,blockRound,validations,deposit,reward,...,lbToggleEma,priority,baker_alias,baker_address,lbEscapeVote,lbEscapeEma,software,software_version,software_date,lbToggle
0,0,3242,BKtiY5Qa7YBbmsWrAbE1GgTnjxM65CxtvW51iJpjQtBbAa...,2018-07-03T00:00:57Z,1,0,0,32,0,0,...,0,0,Foundation baker 3 legacy,tz3RB4aoyjov4KEVRbuhvQ1CKJgBJMWhaeB8,False,0,NaN,NaN,NaN,NaN
1,0,3243,BLu528kh8PCahCS1n5KzdCVuRwCfwgCSA9axy1iMkURDzL...,2018-07-03T00:01:57Z,1,0,0,32,0,0,...,0,0,Foundation baker 7 legacy,tz3WMqdzXqRWXwyvj5Hp2H7QEepaUuS7vd9K,False,0,NaN,NaN,NaN,NaN
2,0,3244,BL2sJe6swL75dKzfFY6bp55BxPqaw4K7Qu1D96xWmDGXEZ...,2018-07-03T00:02:57Z,1,0,0,32,0,0,...,0,0,Foundation Baker 1 legacy,tz3RDC3Jdn4j15J7bBHZd29EUee9gVB1CxD9,False,0,NaN,NaN,NaN,NaN
3,0,3245,BLcGbBGb32Wp2JTUWtZSzuTHb42GXnWuL9WXkiDhoUtdUL...,2018-07-03T00:03:57Z,1,0,0,32,0,0,...,0,0,Foundation baker 7 legacy,tz3WMqdzXqRWXwyvj5Hp2H7QEepaUuS7vd9K,False,0,NaN,NaN,NaN,NaN
4,0,3246,BLS6f6Uf6oXcHR8C5jDBLfeaJcrw7onqaRC37CGBMoaBRw...,2018-07-03T00:04:57Z,1,0,0,32,0,0,...,0,0,Foundation baker 8 legacy,tz3VEZ4k6a4Wx42iyev6i2aVAptTRLEAivNN,False,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3779281,624,3782523,BM7t476icLTEr9LJtXiQMaJY6NqAK3i6JAbJM7nKNZJ5ug...,2023-06-27T23:58:56Z,17,0,0,6940,0,5000000,...,353690635,0,NaN,tz3gLTu4Yxj8tPAcriQVUdxv6BY9QyvzU1az,False,353690635,NaN,v17.1,2023-06-14T12:20:26Z,NaN
3779282,624,3782524,BL32VvzaT8JfVGCmmzDY8CBPzZvkrdv6mj93tF9tnKSN2x...,2023-06-27T23:59:11Z,17,0,0,6936,0,5000000,...,353690635,0,Tezos Foundation Baker 4,tz3UQN6nBQHofmgQ3pZannhiYE2CT7TEZFim,False,353690635,NaN,v17.1,2023-06-14T12:20:26Z,NaN
3779283,624,3782525,BLUmF4K1MrHeqCuTHVidZUsPu2bCxmsKaGQqjdDee8aEV4...,2023-06-27T23:59:26Z,17,0,0,6946,0,5000000,...,353513790,0,Everstake,tz1aRoaRhSpRYvFdyvgWLL6TGyRoGF51wDjM,False,353513790,NaN,v17.1,2023-06-14T12:20:26Z,True
3779284,624,3782526,BKkmcLC3DQ2WfvNSNHpkCojmWbAGxkkEvnHDT931Vx5uU8...,2023-06-27T23:59:41Z,17,0,0,6935,0,5000000,...,354337034,0,Bakery-IL,tz1cYufsxHXJcvANhvS55h3aY32a9BAFB494,True,354337034,NaN,v17.1,2023-06-14T12:20:26Z,False


In [25]:
df_blocks.columns

Index(['cycle', 'level', 'hash', 'timestamp', 'proto', 'payloadRound',
       'blockRound', 'validations', 'deposit', 'reward', 'bonus', 'fees',
       'nonceRevealed', 'proposer_alias', 'proposer_address', 'producer_alias',
       'producer_address', 'lbToggleEma', 'priority', 'baker_alias',
       'baker_address', 'lbEscapeVote', 'lbEscapeEma', 'software',
       'software_version', 'software_date', 'lbToggle'],
      dtype='object')

In [ ]:
drop_cols = ["proto", "payloadRound", "blockRound", 'nonceRevealed', 'proposer_alias', 'producer_alias', 'lbToggleEma', 'priority', 'baker_alias', 'lbEscapeVote', 'lbEscapeEma', \
             'software', 'software_version', 'software_date', 'lbToggle']
df_blocks.drop(drop_cols, axis=1, inplace=True)

In [29]:
df_blocks.to_csv("Tezos/TZ_Blocks.csv", index=False)

## Get cycles

In [30]:
params = {
    "limit" : 10000
}

cycles_url = url_tzkt + "cycles"
i = 0

cycles = []
while(True):
    params["offset"] = 10000 * i
    response = json.loads(requests.get(url = cycles_url, params = params).text)
    
    i += 1
    if len(response) == 0:
        break

    print(len(cycles))
    cycles.extend(response)

0


In [48]:
normalized_cycles = normalize_json_list(cycles)
df_cycles = pd.DataFrame.from_dict(normalized_cycles)
df_cycles.rename({"index":"cycle_index"}, axis=1, inplace=True)

df_cycles = df_cycles.sort_values(by="cycle_index").reset_index(drop=True)

In [50]:
drop_cols = ['snapshotIndex', 'randomSeed']
df_cycles.drop(drop_cols, axis=1, inplace=True)

In [51]:
df_cycles.to_csv("Tezos/TZ_cycles.csv", index=False)

## Get Contracts

In [56]:
params = {
    "limit" : 10000
}

contracts_url = url_tzkt + "contracts"

contracts = []
for i in tqdm(range(0,197942, 10000)):
    params["offset"] = i
    response = json.loads(requests.get(url = contracts_url, params = params).text)
    
    if len(response) == 0:
        break

    contracts.extend(response)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [01:00<00:00,  3.04s/it]


In [61]:
contracts_normalized = normalize_json_list(contracts)
df_contracts = pd.DataFrame.from_dict(contracts_normalized)

In [71]:
drop_cols = ['alias', 'manager_alias', 'delegate_alias', 'delegate_address', \
       'delegate_active', 'delegationLevel', 'manager_publicKey', 'storage', 'creator_alias', 'tzips', 'codeHash', 'typeHash']
df_contracts.drop(drop_cols, axis=1, inplace=True)

In [78]:
df_contracts.to_csv("Tezos/TZ_contract_data.csv", index=False)

## Get Voting Periods

In [13]:
params = {
    "limit" : 10000
}

voting_url = url_tzkt + "voting/periods"

voting_periods = json.loads(requests.get(url = voting_url, params = params).text)

df = pd.DataFrame.from_dict(voting_periods)
df.to_csv(dest_dir + "TZ_voting_periods.csv", index=False)


## Get rewards for bakers and delegators

In [15]:
url_baking_rewards = "https://api.tzkt.io/v1/rewards/bakers/"

with open("/mnt/Ivana/Data/Temp/delegate_addresses.json", "r") as f:
    delegates = json.load(f)

result = []
for a in tqdm(delegates):
    response = json.loads(requests.get(url= url_baking_rewards + a, params=params).text)
    for r in response:
        r["address"] = a
    result.extend(response)

df = pd.DataFrame.from_dict(result)
df.to_csv(dest_dir + "Bakers_rewards.csv", index=False)

100%|██████████| 3219/3219 [05:00<00:00, 10.71it/s]


## Get delegations

In [5]:
LIMIT = 10000
params = {
    "limit" : LIMIT
}

url_del_count = url_tzkt + "operations/delegations/count"
url_delegations = url_tzkt + "operations/delegations"

nr_delegations = int(requests.get(url = url_del_count, params = params).text)



delegations = []
for i in tqdm(range(0, nr_delegations, LIMIT)):
    params["offset"] = i
    response = json.loads(requests.get(url = url_delegations, params = params).text)
    
    if len(response) == 0:
        break

    delegations.extend(normalize_json_list(response))

df_delegations = pd.DataFrame.from_dict(delegations)

100%|██████████| 66/66 [01:33<00:00,  1.41s/it]


In [8]:
df_delegations.to_csv(dest_dir + "TZ_delegations.csv", index=False)

### Get NFTdata

In [3]:
import requests, json

In [ ]:
nft_ids = {
    "Generative Springtime 02": "730111",
    
}

In [50]:
tx_hash = "61058327773184"
url  = "https://api.tzkt.io/v1/operations/" + tx_hash

resp = requests.get(url=url, params=params)

response_js = json.loads(resp.text)
print(json.dumps(response_js, indent=2))


{
  "code": 400,
  "errors": {
    "hash": "Invalid operation hash."
  }
}


In [49]:
hic_et_nunc_addr = "KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton"

zanc = "tz1gBXG9fg8RMDH69KfKqwoTH5sFDmzt5yzm"


jjohn = "tz1gqaKjfQBhUMCE6LhbkpuittRiWv5Z6w38"

url  = "https://api.tzkt.io/v1/tokens/transfers"

params = {
    "limit":100,
    # "token.id": 212197439438849,
    "token.tokenId" : 107012,
    "token.contract": "KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton"
}

resp = requests.get(url=url, params=params)
response_js = json.loads(resp.text)
print(json.dumps(response_js, indent=2))

[
  {
    "id": 55143885176835,
    "level": 1495113,
    "timestamp": "2021-05-31T09:59:30Z",
    "token": {
      "id": 55143885176833,
      "contract": {
        "alias": "hic et nunc NFTs",
        "address": "KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton"
      },
      "tokenId": "107012",
      "standard": "fa2",
      "totalSupply": "1",
      "metadata": {
        "name": "Collection: Generative Monochrome | Name: Birth iii",
        "tags": [
          "generative",
          "axidraw",
          "generativeart",
          "pixart",
          "fractal",
          "algorithmicart",
          "monochrome"
        ],
        "symbol": "OBJKT",
        "formats": [
          {
            "uri": "ipfs://QmZLngz61Ez8qbEmMiPrbG3w6fBYBBVJ4CsMfZ4rGRn3MH",
            "mimeType": "image/png"
          }
        ],
        "creators": [
          "tz1gVKxpEGC7QW1fZyEEMEW2kgJRbpgWLNpD"
        ],
        "decimals": "0",
        "displayUri": "ipfs://QmdqW4NU5VMEjU69aWy82sgXu1bBFRj7ajjs5ksmytw7

In [60]:
url = "https://api.tzkt.io/v1/operations/transactions"
params = {
    "id": 66215448936448
}

resp2 = requests.get(url=url, params=params)
response_js2 = json.loads(resp2.text)
print(len(response_js2))
print(json.dumps(response_js2, indent=2))

1
[
  {
    "type": "transaction",
    "id": 66215448936448,
    "level": 1583630,
    "timestamp": "2021-08-02T10:45:34Z",
    "block": "BLf2RVMQ5XCgfwG5yRpBjpyUCh7sYDNWFqGkwb3UwA8uKxh1zWn",
    "hash": "oouZGcbSL7qxXCng21Qv4PEbRzQugTdKQPH93p9NMJzCGw43BJJ",
    "counter": 11436175,
    "initiator": {
      "alias": "Shvembldr",
      "address": "tz1WqBtWEXjYPwKofxDPsf2V8Jo7dRxdKYqu"
    },
    "sender": {
      "alias": "hic et nunc Marketplace",
      "address": "KT1HbQepzV1nVGg8QVznG7z4RcHseD5kwqBn"
    },
    "senderCodeHash": 53636017,
    "nonce": 12,
    "gasLimit": 0,
    "gasUsed": 26674,
    "storageLimit": 0,
    "storageUsed": 67,
    "bakerFee": 0,
    "storageFee": 16750,
    "allocationFee": 0,
    "target": {
      "alias": "hic et nunc NFTs",
      "address": "KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton"
    },
    "targetCodeHash": 1973375561,
    "amount": 0,
    "parameter": {
      "entrypoint": "transfer",
      "value": [
        {
          "txs": [
            {
     

In [64]:
tx_hash = "oouZGcbSL7qxXCng21Qv4PEbRzQugTdKQPH93p9NMJzCGw43BJJ"
url  = "https://api.tzkt.io/v1/operations/" + tx_hash

resp = requests.get(url=url, params=params)

response_js = json.loads(resp.text)
print(len(response_js))
print(json.dumps(response_js, indent=2))
# for tx in response_js:
#     if float(tx["amount"]) > 0:
#         print(json.dumps(tx, indent=2))

5
[
  {
    "type": "transaction",
    "id": 66215444742144,
    "level": 1583630,
    "timestamp": "2021-08-02T10:45:34Z",
    "block": "BLf2RVMQ5XCgfwG5yRpBjpyUCh7sYDNWFqGkwb3UwA8uKxh1zWn",
    "hash": "oouZGcbSL7qxXCng21Qv4PEbRzQugTdKQPH93p9NMJzCGw43BJJ",
    "counter": 11436175,
    "sender": {
      "alias": "Shvembldr",
      "address": "tz1WqBtWEXjYPwKofxDPsf2V8Jo7dRxdKYqu"
    },
    "gasLimit": 87398,
    "gasUsed": 56363,
    "storageLimit": 350,
    "storageUsed": 0,
    "bakerFee": 9023,
    "storageFee": 0,
    "allocationFee": 0,
    "target": {
      "alias": "hic et nunc Marketplace",
      "address": "KT1HbQepzV1nVGg8QVznG7z4RcHseD5kwqBn"
    },
    "targetCodeHash": 53636017,
    "amount": 200000000,
    "parameter": {
      "entrypoint": "collect",
      "value": "501652"
    },
    "storage": {
      "fee": "25",
      "objkt": "KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton",
      "swaps": 6072,
      "counter": "640775",
      "manager": "tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrG